# API de Gestión de Tareas

Este proyecto es una API RESTful desarrollada con **Flask** y **Flask-JWT-Extended**. Permite gestionar tareas de manera segura con autenticación basada en JWT. Los usuarios pueden registrarse, iniciar sesión y crear, ver y eliminar tareas asociadas a su cuenta.

## Funcionalidades
- **Registro de usuarios**: Los usuarios pueden crear una cuenta proporcionando un nombre de usuario y una contraseña.
- **Inicio de sesión**: Los usuarios pueden iniciar sesión y obtener un token JWT para autenticar sus solicitudes.
- **Gestión de tareas**: Los usuarios autenticados pueden crear, listar y eliminar tareas.

Este notebook explica las librerías utilizadas, las estructuras de datos y la implementación del código.
## Librerías

### `flask 3.1.0`
Flask es un micro-framework de Python para desarrollar aplicaciones web. En este proyecto se utiliza para construir la API.

```python
from flask import Flask, jsonify, request
```
### `flask-jwt-extended 4.6.0`
Libreria que se utiliza para manejar la autenticación con tokens JWT. Permite crear, verificar y gestionar tokens de acceso.

```python
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
```
### `flask-sqlalchemy`
Libreria que permite interactuar con la base de datos mediante objetos Python en lugar de escribir SQL directamente.

```python
from models import db, User, Task
```

# ¡Es importante! no olvidar para este proyecto, generar un archivo llamado "requirements.txt" donde se alojen las librerias de la sigueinte forma:
## En el archivo de requirements colocar

### flask==3.1.0
### flask-jwt-extended==4.6.0
### flask-sqlalchemy

## Y correr el archivo, en terminal o consola con la siguiente instruccion 
```
pip install -r requirements.txt
```
De esta manera, instalados los requirements, ya podemos hacer uso de las librerias en este proyecto

In [ ]:
# Librerias importadas en el archivo principal de la aplicación
from flask import Flask, jsonify, request
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from models import db, User, Task

In [ ]:
# Mencionamos que este proyecto utilizaria a flask, asi que se menciona al inicio 
app = Flask(__name__)

# Elementos de SQLite, que nos permite dirigir los datos guardados a una base de datos llamada "database.db" y nos permite configurar la prioridad
# Que los datos encontraran en la tabla de base de datos asociada, ademas de generar una funcion que permitira mas adelante crear un token de seguridad
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['JWT_SECRET_KEY'] = 'seckey'

db.init_app(app)
jwt = JWTManager(app)


with app.app_context():
    db.create_all()

## Estructuras de Datos

### `User` y `Task`
Se utilizan dos modelos: `User` para gestionar los usuarios y `Task` para gestionar las tareas. Ambas son clases de SQLAlchemy que se mapearán a tablas en la base de datos.

- **User**: Contiene información de los usuarios, como su nombre de usuario y contraseña.
- **Task**: Contiene información sobre las tareas, como el título y el usuario asociado.

### Base de Datos
La base de datos utilizada es **SQLite**, configurada con SQLAlchemy.


```python
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
```

## Rutas y Funcionalidades

### Ruta `/register` (POST)
Registra a un nuevo usuario en la base de datos.

**Entrada**:
- `username`: El nombre de usuario.
- `password`: La contraseña del usuario.

**Salida**:
- Mensaje de éxito si el registro es exitoso, o error si el usuario ya existe.

```python
@app.route('/register', methods=['POST'])
def register():
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')

    if User.query.filter_by(username=username).first():
        return jsonify({'error': 'El usuario ya existe'}), 400

    user = User(username=username)
    user.set_password(password)
    db.session.add(user)
    db.session.commit()
    return jsonify({'message': 'Usuario registrado exitosamente'}), 201
```

### Ruta `/login` (POST)
Inicia sesión y devuelve un token JWT para autenticarse en rutas protegidas.

**Entrada**:
- `username`: El nombre de usuario.
- `password`: La contraseña del usuario.

**Salida**:
- Un token JWT si las credenciales son correctas.

```python
    @app.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    user = User.query.filter_by(username=data['username']).first()

    if not user or not user.check_password(data['password']):
        return jsonify({'error': 'Credenciales inválidas'}), 401

    access_token = create_access_token(identity=user.id)
    return jsonify({'access_token': access_token}), 200
```

### Ruta `/task` (GET ,POST)
Inicia sesión y devuelve un token JWT para autenticarse en rutas protegidas.

- GET: Devuelve todas las tareas del usuario autenticado.
- POST: Crea una nueva tarea para el usuario autenticado.
    
```python
    @app.route('/tasks', methods=['GET', 'POST'])
@jwt_required()
def handle_tasks():
    user_id = get_jwt_identity()

    if request.method == 'GET':
        tasks = Task.query.filter_by(user_id=user_id).all()
        return jsonify([{'id': task.id, 'title': task.title} for task in tasks]), 200

    if request.method == 'POST':
        data = request.get_json()
        new_task = Task(title=data['title'], user_id=user_id)
        db.session.add(new_task)
        db.session.commit()
        return jsonify({'message': 'Tarea creada'}), 201
```

### Ruta `/tasks/<task_id>` (DELETE)
Elimina la tarea del usuario autenticado.

```python
@app.route('/tasks/<int:task_id>', methods=['DELETE'])
@jwt_required()
def delete_task(task_id):
    user_id = get_jwt_identity()
    task = Task.query.filter_by(id=task_id, user_id=user_id).first()

    if not task:
        return jsonify({'error': 'Tarea no encontrada'}), 404

    db.session.delete(task)
    db.session.commit()
    return jsonify({'message': 'Tarea eliminada'}), 200
```

## Conclusión

Este proyecto proporciona una API de gestión de tareas segura mediante JWT. Los usuarios pueden crear una cuenta, iniciar sesión y gestionar tareas pendientes de manera sencilla.

Posibles mejoras incluyen la implementación de funciones de actualización de tareas y una interfaz de usuario más robusta.
